In [1]:
from database.database.engine import init_db
from constants import CONN_STRING
from database.database.ask_db import *
from database.operations.fen import *

/home/jon/anaconda3/envs/chessism_gpu/lib/python3.12/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


# Connection to the DB

In [2]:
await init_db(CONN_STRING)

Database 'chessism' already exists.
Ensuring database tables exist...
Database tables checked/created.
Asynchronous database initialization complete.


# In cas you need to erase all tables
it will only erase tables from this API: 'fen','game_fen_association'

In [3]:
#await delete_all_leela_tables()

# In case you need reset every game.fens_done to False

In [4]:
#await reset_all_game_fens_done_to_false()

### In case you want to set:  
every game.fens_done to False but just for one player:

In [5]:
#await reset_player_game_fens_done_to_false('vi_pranav')

### Pipeline to analyze fens that are repeated more_than int times

In [6]:
start = time.time()

In [7]:
analize_results = await analize_most_repeated_fens(between = (5,20), limit= 10)
to_insert = await format_leela_results(analize_results)
await DBInterface(Fen).update_fen_analysis_data(to_insert)

...Starting Leela engine...


<UciProtocol (pid=72119)>: stderr >>        _
<UciProtocol (pid=72119)>: stderr >> |   _ | |
<UciProtocol (pid=72119)>: stderr >> |_ |_ |_| v0.31.2 built Oct 20 2024


...Leela engine ready...


<UciProtocol (pid=72119)>: stderr >> Loading weights file from: 791556.pb.gz
<UciProtocol (pid=72119)>: stderr >> Creating backend [cuda-fp16]...
<UciProtocol (pid=72119)>: stderr >> CUDA Runtime version: 11.1.0
<UciProtocol (pid=72119)>: stderr >> Latest version of CUDA supported by the driver: 12.9.0
<UciProtocol (pid=72119)>: stderr >> GPU: NVIDIA GeForce RTX 4060
<UciProtocol (pid=72119)>: stderr >> GPU memory: 7.99548 Gb
<UciProtocol (pid=72119)>: stderr >> GPU clock frequency: 2490 MHz
<UciProtocol (pid=72119)>: stderr >> GPU compute capability: 8.9
<UciProtocol (pid=72119)>: stderr >> L2 cache capacity: 25165824



--- Analysis Complete ---
Successfully updated 10 FEN records with analysis data.


10

In [8]:
end = time.time()
print('enden in ',end-start)

enden in  19.370742082595825


### Doing the pipeline in a loop
So the 3d card can lower the temp.

In [9]:
start_analysis_loop = time.time() 
n_repetitions = 3
between = (2,4)
limit = 10
min_wait_time_seconds = 20
max_wait_time = 1/4 #quarter of the time it take to run one loop
verbose_each = 100
analyse_time_limit = 1.5
nodes_limit = 100_000

In [10]:
for ind, batches in enumerate(range(n_repetitions)):
    start_batch = time.time()
    print(f'{batches} out of {n_repetitions}')
    last_loop = list(range(n_repetitions))[-1]
    
    analize_results = await analize_most_repeated_fens(between = between,
                                                       limit= limit,
                                                      verbose_each = verbose_each,
                                                      analyse_time_limit = analyse_time_limit,
                                                      nodes_limit = nodes_limit)
    to_insert = await format_leela_results(analize_results)
    await DBInterface(Fen).update_fen_analysis_data(to_insert)

    print(f'{ind} batch done in: ', time.time()-start_batch)
    print('#####')
    print('... Cool of waiting ...')
    print('#####')
    if batches == last_loop:
        print('Done everything in: ', time.time()-start)
        break
    sleep_for = max(min_wait_time_seconds, (time.time()-start_batch) * max_wait_time)
    print(f'... sleeping for {sleep_for} seconds')
    time.sleep(sleep_for)
    print('#####')
    print('... End of waiting Cool ...')
    print('#####')

0 out of 3
...Starting Leela engine...


<UciProtocol (pid=74415)>: stderr >>        _
<UciProtocol (pid=74415)>: stderr >> |   _ | |
<UciProtocol (pid=74415)>: stderr >> |_ |_ |_| v0.31.2 built Oct 20 2024


...Leela engine ready...


<UciProtocol (pid=74415)>: stderr >> Loading weights file from: 791556.pb.gz
<UciProtocol (pid=74415)>: stderr >> Creating backend [cuda-fp16]...
<UciProtocol (pid=74415)>: stderr >> CUDA Runtime version: 11.1.0
<UciProtocol (pid=74415)>: stderr >> Latest version of CUDA supported by the driver: 12.9.0
<UciProtocol (pid=74415)>: stderr >> GPU: NVIDIA GeForce RTX 4060
<UciProtocol (pid=74415)>: stderr >> GPU memory: 7.99548 Gb
<UciProtocol (pid=74415)>: stderr >> GPU clock frequency: 2490 MHz
<UciProtocol (pid=74415)>: stderr >> GPU compute capability: 8.9
<UciProtocol (pid=74415)>: stderr >> L2 cache capacity: 25165824



--- Analysis Complete ---
Successfully updated 10 FEN records with analysis data.
0 batch done in:  15.112958908081055
#####
... Cool of waiting ...
#####
... sleeping for 20 seconds
#####
... End of waiting Cool ...
#####
1 out of 3
...Starting Leela engine...


...


--- Analysis Complete ---
Successfully updated 10 FEN records with analysis data.
1 batch done in:  13.772090911865234
#####
... Cool of waiting ...
#####
... sleeping for 20 seconds
#####
... End of waiting Cool ...
#####
2 out of 3


<UciProtocol (pid=79010)>: stderr >>        _
<UciProtocol (pid=79010)>: stderr >> |   _ | |
<UciProtocol (pid=79010)>: stderr >> |_ |_ |_| v0.31.2 built Oct 20 2024


...Starting Leela engine...
...Leela engine ready...


<UciProtocol (pid=79010)>: stderr >> Loading weights file from: 791556.pb.gz
<UciProtocol (pid=79010)>: stderr >> Creating backend [cuda-fp16]...
<UciProtocol (pid=79010)>: stderr >> CUDA Runtime version: 11.1.0
<UciProtocol (pid=79010)>: stderr >> Latest version of CUDA supported by the driver: 12.9.0
<UciProtocol (pid=79010)>: stderr >> GPU: NVIDIA GeForce RTX 4060
<UciProtocol (pid=79010)>: stderr >> GPU memory: 7.99548 Gb
<UciProtocol (pid=79010)>: stderr >> GPU clock frequency: 2490 MHz
<UciProtocol (pid=79010)>: stderr >> GPU compute capability: 8.9
<UciProtocol (pid=79010)>: stderr >> L2 cache capacity: 25165824



--- Analysis Complete ---
Successfully updated 10 FEN records with analysis data.
2 batch done in:  10.08233904838562
#####
... Cool of waiting ...
#####
Done everything in:  103.94300556182861


In [18]:
end = time.time()
print('enden in ',end-start)

enden in  22585.197417020798


In [19]:
print(end)

1752028448.9136944


In [8]:
done, lack = await get_fen_score_counts()

In [9]:
print(f'done: {done:,}')
print(f'still: {lack:,}')

done: 166,854
still: 119,635


In [64]:
print(f'done: {done:,}')
print(f'still: {lack:,}')

done: 70,844
still: 157,434


In [ ]:
# more than 10

In [61]:
print(f'done: {done:,}')
print(f'still: {lack:,}')

done: 61,814
still: 49,714


In [ ]:
45623/65905

In [ ]:
sql_null = "SELECT COUNT(fen) FROM fen WHERE score IS NULL AND n_games > 20;"
result_null = await open_async_request(sql_null)

In [ ]:
result_null

In [ ]:
18180/49627184

In [ ]:
#######   lksdghfkldghfjksdghf

In [ ]:
await get_fen_score_counts()

In [ ]:
await get_fen_score_counts()

In [ ]:
await get_fen_score_counts()

In [ ]:
20190/34929

In [ ]:
'gkdfkkdk'

In [ ]:
'DDDDDDDDDDDDDDDDDDDDDDOOOOOOOOOOOOOOOOONENENENEN'

In [ ]:
'DDDDDDDDDDDDDDDDDDDDDDOOOOOOOOOOOOOOOOONENENENEN'

In [ ]:
'DDDDDDDDDDDDDDDDDDDDDDssjkk sksksk'

In [ ]:

#await insert_every_main_character(limit_to_n_players = False)

In [ ]:
some_foo_process()
sound_alarm()